In [2]:
import torch

from models import MambaModel
from models import TransformerModelLooped
from curriculum import CurriculumSimple
from train import train_without_config, validate_model

n_dims = 10

In [3]:
model_ssm = MambaModel(n_dims=n_dims,
                       n_positions=101,
                       n_embd=256,
                       n_layer=1,
                       pred_type="regression").cuda()

cirriculum_ssm = CurriculumSimple(n_dims, 31, 5, [5000, n_dims, 0], [5000, 31, 0], [1000, 5, 1])

metrics_l1_ssm = train_without_config(
    model_ssm, cirriculum_ssm, model_n_dims=n_dims,
    log_every_steps=10, train_steps=100, family="ssm_gpt2",
    do_wandb_log=False, seed=None, task_name="linear_regression", device="cuda")

number of parameters: 0.44M


loss 9.946172714233398: 100%|██████████| 100/100 [01:38<00:00,  1.02it/s]
